In [8]:
import re, json, html
import pandas as pd
import numpy as np

def strip_tags(s: str) -> str:
    s = re.sub(r'<br\s*/?>', '\n', s, flags=re.I)
    s = re.sub(r'<[^>]+>', '', s)
    s = html.unescape(s)
    # normalize spaces
    s = re.sub(r'\xa0', ' ', s)  # non-breaking
    s = re.sub(r'[ \t]+', ' ', s).strip()
    return s

def to_number(x):
    x = x.strip()
    if x in {'.', '. .', ''}:
        return None
    # remove commas in thousands
    x = x.replace(',', '')
    try:
        if '.' in x:
            return float(x)
        return int(x) if x.isdigit() else x
    except ValueError:
        return x

with open("data/USCODE22_LLCP_102523.HTML", "r", encoding="utf-8", errors="ignore") as f:
    snippet = f.read()
    
table_blocks = re.findall(r'(<table class="table".*?</table>)', snippet, flags=re.S|re.I)


items = []
for tb in table_blocks:
    # header info
    m = re.search(r'<td[^>]*class="[^"]*linecontent[^"]*"[^>]*colspan="5"[^>]*>(.*?)</td>', tb, flags=re.S|re.I)
    meta = {}
    if m:
        meta_text = strip_tags(m.group(1))
        # Split on newlines and parse key: value
        for line in meta_text.split('\n'):
            if ':' in line:
                k, v = line.split(':', 1)
                meta[k.strip()] = v.strip()
    # body rows
    body = re.search(r'<tbody>(.*?)</tbody>', tb, flags=re.S|re.I)
    rows = []
    if body:
        for tr in re.findall(r'<tr>(.*?)</tr>', body.group(1), flags=re.S|re.I):
            tds = re.findall(r'<td[^>]*>(.*?)</td>', tr, flags=re.S|re.I)
            if len(tds) == 5:
                value = strip_tags(tds[0])
                vlabel = strip_tags(tds[1])
                freq = strip_tags(tds[2])
                perc = strip_tags(tds[3])
                wperc = strip_tags(tds[4])
                rows.append({
                    "value": value,
                    "label": vlabel,
                    "frequency": to_number(freq),
                    "percentage": to_number(perc),
                    "weighted_percentage": to_number(wperc),
                })
    if meta or rows:
        items.append({
            "sas_variable": meta.get("SAS Variable Name"),
            "label": meta.get("Label"),
            "section": meta.get("Section Name"),
            "module_number": meta.get("Module Number"),
            "question_number": meta.get("Question Number"),
            "column": meta.get("Column"),
            "type": meta.get("Type of Variable"),
            "question": meta.get("Question"),
            "raw_meta": meta,
            "categories": rows
        })

# Save to JSON file for download
json_path = "data/codebook_parsed.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(items, f, ensure_ascii=False, indent=2)

# Build a convenient DataFrame for querying: one row per (variable, value)
records = []
for item in items:
    for c in item["categories"]:
        records.append({
            "sas_variable": item["sas_variable"],
            "variable_label": item["label"],
            "value": c["value"],
            "value_label": c["label"],
            "frequency": c["frequency"],
            "percentage": c["percentage"],
            "weighted_percentage": c["weighted_percentage"],
        })

df = pd.DataFrame.from_records(records)

json_path, len(items), df.shape


('data/codebook_parsed.json', 324, (2011, 7))

In [26]:
import json
data = pd.read_csv("data/train.csv")
with open("data/codebook_parsed.json") as f:
    dict = json.load(f)

In [ ]:
cols = data.columns.tolist()
cols = cols[242:]

keep_cols = ["_RFHLTH", "_PHYS14D", "_MENT14D", "_HLTHPLN", "_HCVU652", "_TOTINDA", "_LTASTH1", "_CASTHM1", "_ASTHMS1",
             "_SEX", "_AGEG5YR", "_SMOKER3", "_CURECI2", "_PACKDAY", "_SMOKGRP", "_LCSREC", "_RFDRHV8", "_FLSHOT7","_PNEUMO3",
             "ID", "TARGET"]
remove_cols = ["QSTLANG", "_METSTAT","_URBSTAT", "MSCODE", "_STSTR", "_STRWT", "_RAWRAKE", "_WT2RAKE", "_MRACE2", "_IMPRACE",
             "_IMPRACE","_CHISPNC", "_CRACE2", "_CPRACE2", "CAGEG", "_CLLCPWT", "_DUALUSE","_PRACE2", "_HISPANC","_RACE1",
             "_DUALCOR", "_LLCPWT", "_LLCPWT2", "_EXTETH3", "_ALTETH3", "_DENVST3", "_DRDXAR2", "_RACEG22", "_RACEGR4", "_RACEPR1",
             "_AGE65YR", "_AGE80", "_AGE_G", "HTIN4", "HTM4","WTKG3", "WTKG3", "_BMI5", "_RFBMI5","_BMI5CAT", "_CHLDCNT", "_EDUCAG",
             "_INCOMG1", "_RFMAM22", "_MAM5023", "_HADCOLN", "_CLNSCP1","_HADSIGM", "_SGMSCP1", "_SGMS101", "_RFBLDS5", "_STOLDN1",
             "_VIRCOL1", "_SBONTI1", "_CRCREC2", "_RFSMOK3", "_YRSSMOK", "_PACKYRS", "_YRSQUIT", "DRNKANY6", "DROCDY4_", "_RFBING6",
             "_DRNKWK2", "_AIDTST4"]
tbd_cols = []
blank_list = []

for col in cols:
    if col not in keep_cols and col not in remove_cols and col not in tbd_cols:
        print(f"Variable: {col}")
        entry = next((item for item in dict if item["sas_variable"] == col), None)
        if entry:
            total = 0
            print(f"Label: {entry['label']}")
            print(f"Question: {entry['question']}")
            print("Categories:")
            for cat in entry["categories"]:
                print(f"  Value: {cat['value']}, Label: {cat['label']}, Freq: {cat['frequency']}, Perc: {cat['percentage']}")
                total += cat['frequency'] if cat['frequency'] is not None else 0

            if "BLANK" in [cat['value'] for cat in entry["categories"]]:
                blank_cat = next(cat for cat in entry["categories"] if cat['value'] == "BLANK")
                percent_blank = blank_cat["percentage"]

                if percent_blank is not None and percent_blank > 80:
                    blank_list.append(col)
                    print(f"  --> Added to blank_list (percent blank: {percent_blank}%)")
        else:
            print("No dictionary entry found.")
        print("\n")




None
